# Leitura dos dados

In [188]:
import pandas as pd

#leitura dos dados
data = pd.read_csv("dados\\bank.csv",sep=";")


data['y'] = pd.Series(np.where(data.y.values == 'no', 0, 1),data.index)
y = data['y']
X = data.drop(columns=['y'])
print(len(X.columns))

16


# Seleção de features



## 1) Retirada de valores nulos

Retirada de variáveis com uma grande percentual de valores nulos dos quais iremos considerar o corte como 20%.


In [189]:
# saving missing values in a variable
a = X.isnull().sum()/len(X)*100
# saving column names in a variable
variables = X.columns
variable = [ ]
for i in range(0,12):
    if a[i]>=20:   #setting the threshold as 20%
        variable.append(variables[i])
X = X.drop(columns=variable)

print(len(X.columns))

16


## 2) Preencher nulos com a média

As colunas que foram do tipo numérica terão seus valores nulos preenchidos pela média

In [190]:
X.fillna(X.median(), inplace=True)

## 3) Filtro de variância

Variáveis que apresentarem variância abaixo de 10% serão retiradas também.

In [191]:
var = X.var()
numeric = X.columns
variable = [ ]
for i in range(0,len(var)):
    if var[i]<=10:   #setting the threshold as 10%
       variable.append(numeric[i+1])
X = X.drop(columns=variable)
print(len(X.columns))

14


## 4) Filtro de correlação



Variáveis que apresentarem correlação acima de 0.6, uma delas será retirada.


In [192]:
matrix = np.array(X.corr().abs())
np.where((matrix >0.5)&(matrix <1))[0]




array([4, 5], dtype=int64)

In [193]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=1, max_depth=10)
X=pd.get_dummies(X)
model.fit(X,y)

features = X.columns
importances = model.feature_importances_
indices = np.argsort(importances)[-9:]  # top 10 features

print(len(X.columns))
from sklearn.feature_selection import SelectFromModel
feature = SelectFromModel(model)
X = feature.fit_transform(X, y)
len(X)

C:\Users\caico\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


48


4521

# Agrupamento dos dados numa unica categoria


In [127]:
import numpy as np

#agrupa os basics numa unica categoria
data['education']=np.where(data['education'] =='basic.9y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'Basic', data['education'])

data['y'] = pd.Series(np.where(data.y.values == 'no', 0, 1),data.index)


C:\Users\caico\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


# Criação de variáveis dummy

In [128]:
#essas são as variáveis categoricas
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']

for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

KeyError: 'day_of_week'

In [ ]:

#separa as variaveis da variavel resposta
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']

# Balanceamento
Para balancear iremos utilizar a técnica SMOTE do qual cria amostras parecidas com as outras.

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

In [ ]:
data_final_vars=data_final.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]


from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
y

In [ ]:
cols=['euribor3m', 'job_blue-collar', 'job_housemaid', 'marital_unknown', 'education_illiterate', 'default_no', 'default_unknown', 
      'contact_cellular', 'contact_telephone', 'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 
      'month_may', 'month_nov', 'month_oct', "poutcome_failure", "poutcome_success"] 

X=os_data_X[cols]

y=os_data_y['y']

In [ ]:
import statsmodels.api as sm


logit_model=sm.Logit(y.astype(float),X.astype(float))
result=logit_model.fit()
print(result.summary2())